# Comparison with pennsignals BayesCHIME

Source: https://github.com/pennsignals/chime_sims

In [ ]:
from os import getcwd, path

from scipy.stats import beta, gamma, norm
from scipy.optimize import curve_fit

import pandas as pd
from numpy import linspace, sqrt

from plotly.subplots import make_subplots
from plotly.graph_objects import Scatter

In [ ]:
DATA = path.join(path.abspath(getcwd()), "data", "penn")

In [ ]:
location = "Downtown"

parameters = pd.read_csv(path.join(DATA, f"{location}_parameters.csv"))
data = (
    pd.read_csv(path.join(DATA, f"{location}_ts.csv"), parse_dates=["date"])
    .dropna(how="all", axis=1)
    .fillna(0)
    .set_index("date")
    .astype(int)
)

parameters

In [ ]:
PARAMETER_MAP = {
    "n_hosp": "hospitalized_initial",
    "hosp_prop": "hospitalization_rate",
    # "hosp_LOS": "recovery_days_i",
    "recovery_days": "recovery_days_i",
    "logistic_k": "social_distance_halfing_days",
    "logistic_x0": "social_distance_delay",
    "logistic_L": "ratio",
    "beta": "beta_i",
}

In [ ]:
DIST_MAP = {
    "beta": beta,
    "gamma": gamma,
}


def get_dist_df(distribution, p1, p2, **kwargs):
    """
    """

    data = {}
    args = {"a": p1}
    args["b" if distribution == "beta" else "scale"] = p2

    dist = DIST_MAP[distribution](**args)
    x = linspace(dist.ppf(0.01), dist.ppf(0.99), 100)
    y = dist.pdf(x)
    mean, var = dist.stats(moments="mv")

    data = pd.DataFrame({"x": x, "y": y})
    data["dist"] = distribution
    data["param"] = kwargs.get("param")

    return data, dist

In [ ]:
def fit_norm_dist(x, y, dist, param):
    mean = dist.stats("m")
    scale = sqrt(dist.stats("v"))
    mean, scale = curve_fit(norm.pdf, xdata=x, ydata=y, p0=(mean, scale))[0]

    dist = norm(mean, scale)
    y = dist.pdf(x)
    data = pd.DataFrame({"x": x, "y": y})
    data["dist"] = "normal"
    data["param"] = param

    return data, dist

In [ ]:
dfs = []
mapped_pars = {}

for idx, row in parameters.iterrows():

    if row["param"] not in PARAMETER_MAP:
        continue

    mapped_param = PARAMETER_MAP[row["param"]]

    if row["distribution"] == "constant":
        pass

    else:
        tmp_df, dist = get_dist_df(**row)
        dfs.append(tmp_df)

        tmp_df, dist = fit_norm_dist(tmp_df.x, tmp_df.y, dist, row["param"])
        dfs.append(tmp_df)


df = pd.concat(dfs, ignore_index=True)
df.head()

In [ ]:
params = df.param.unique()

col_wrap = 4
n_cols = min(col_wrap, len(params))
n_rows = len(params) // 4 + 1

fig = make_subplots(
    cols=n_cols,
    rows=n_rows,
    shared_xaxes=False,
    shared_yaxes=False,
    subplot_titles=params,
)

for i, param in enumerate(params):
    tmp_df = df.query("param == @param")
    for color, dist in zip(["blue", "green"], tmp_df.dist.unique()):
        ttmp_df = tmp_df.query("dist == @dist")
        fig.add_trace(
            Scatter(
                x=ttmp_df.x,
                y=ttmp_df.y,
                name=dist,
                line_color=color,
                mode="lines" if dist == "normal" else "markers",
                showlegend=False,
            ),
            col=i % col_wrap + 1,
            row=i // col_wrap + 1,
        )

fig.show()